In [9]:
import numpy as np
import csv
import pandas as pd

In [2]:
# import data from csv into dataframe
crash_data = pd.read_csv('Vehicular_Crash_Data.csv')
#show the data
crash_data.head()

/Users/simranjitsachar/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,﻿X,Y,OBJECTID,COMPLAINTNUMBER,ACCIDENTDATE,YEAR,DATEENTERED,MILITARYTIME,DAYOFWEEK,REPORTDATE,...,REVISED_QUADRANT,RECORDREMA,LASTUPDATE,ENTEREDBY,LATITUDE,LONGITUDE,IMPAIRMENT,STREETSEGID,TAXIINVOLVED,TOURBUSINVOLVED
0,-77.097015,38.947120,445,12076031,01-JUN-12,2012,04-FEB-14,1254,Friday,02-JUN-12,...,1,"NEW,LVL:2",04-FEB-14,MPD,38.947113,-77.097013,NaN,2356,NaN,NaN
1,-77.097009,38.919510,446,13167289,20-NOV-13,2013,20-NOV-13,1626,Wednesday,20-NOV-13,...,1,"NEW,LVL:2",20-NOV-13,MPD,38.919502,-77.097007,NaN,7198,NaN,NaN
2,-77.096995,38.943390,447,12057272,27-APR-12,2012,27-APR-12,1615,Friday,27-APR-12,...,1,"NEW,LVL:2",27-APR-12,MPD,38.943382,-77.096992,NaN,10734,NaN,NaN
3,-77.096987,38.915828,448,15023562,17-FEB-15,2015,24-FEB-15,945,Tuesday,17-FEB-15,...,1,"NEW,LVL:2",24-FEB-15,MPD,38.915820,-77.096985,NaN,3429,NaN,NaN
4,-77.096949,38.918073,449,11061645,04-MAY-11,2011,09-MAY-11,2030,Wednesday,04-MAY-11,...,1,"NEW,LVL:1",09-MAY-11,MPD,38.918065,-77.096947,NaN,927,NaN,NaN


In [11]:
# classify military time
crash_data['Time_cat'] = "NA"
def func(row):
    if row['MILITARYTIME'] >= 0 and row['MILITARYTIME'] < 0700:
        return 'Overnight'
    elif row['MILITARYTIME'] >= 0700 and row['MILITARYTIME'] < 1000:
        return 'AM_rush'
    elif row['MILITARYTIME'] >= 1000 and row['MILITARYTIME'] < 1200:
        return 'AM_norush'
    elif row['MILITARYTIME'] >= 1200 and row['MILITARYTIME'] < 1600:
        return 'PM_norush'
    elif row['MILITARYTIME'] >= 1600 and row['MILITARYTIME'] < 1800:
        return 'AM_norush'
    elif row['MILITARYTIME'] >= 1800 and row['MILITARYTIME'] < 2200:
        return 'Evening'
    elif row['MILITARYTIME'] >= 2200:
        return 'Overnight'
    else:
        return 'NoTime'
crash_data['Time_cat'] = crash_data.apply(func, axis=1)

In [24]:
# clean weather by making 'others' as consolidated value for junk data 
def func(row):
    if row['WEATHER'] == 'Rain':
         return 'Rain'
    elif row['WEATHER'] == 'Clear':
        return 'Clear'
    elif row['WEATHER'] == 'Snow':
        return 'Snow'
    elif row['WEATHER'] == 'Fog/Mist':
        return 'Fog/Mist'
    elif row['WEATHER'] == 'Severe Crosswind':
        return 'Severe Crosswind'
    else:
        return 'others'
crash_data['weather_cat'] = crash_data.apply(func, axis=1)

In [13]:
# showing the data
crash_data[['Time_cat','MILITARYTIME','WEATHER']].head()

,Time_cat,MILITARYTIME,WEATHER
0,PM_norush,1254,Rain
1,AM_norush,1626,Clear
2,AM_norush,1615,Clear
3,AM_rush,945,Snow
4,Evening,2030,Clear


In [26]:
# link between time and weather - getting the counts for source and destination linkage 
time_weather = crash_data[['STREETSEGID','Time_cat','weather_cat']].groupby(['Time_cat','weather_cat']).count().reset_index()
time_weather.head()

,Time_cat,weather_cat,STREETSEGID
0,AM_norush,Clear,20451
1,AM_norush,Fog/Mist,157
2,AM_norush,Rain,2404
3,AM_norush,Severe Crosswind,67
4,AM_norush,Snow,400


In [42]:
time_weather.columns = ['Time_cat','weather_cat','count']
print time_weather
# export dataframe to csv
time_weather.to_csv('time_weather.csv')

     Time_cat       weather_cat  count
0   AM_norush             Clear  20451
1   AM_norush          Fog/Mist    157
2   AM_norush              Rain   2404
3   AM_norush  Severe Crosswind     67
4   AM_norush              Snow    400
5   AM_norush            others   1496
6     AM_rush             Clear  16006
7     AM_rush          Fog/Mist    289
8     AM_rush              Rain   2032
9     AM_rush  Severe Crosswind     58
10    AM_rush              Snow    395
11    AM_rush            others   1409
12    Evening             Clear  14753
13    Evening          Fog/Mist    183
14    Evening              Rain   2336
15    Evening  Severe Crosswind     52
16    Evening              Snow    264
17    Evening            others   1449
18  Overnight             Clear  15031
19  Overnight          Fog/Mist    254
20  Overnight              Rain   1960
21  Overnight  Severe Crosswind     52
22  Overnight              Snow    301
23  Overnight            others   1375
24  PM_norush            

In [35]:
# null values or spaces
def func(row):
    if row['ACCIDENTTYPE'] == '' or row['ACCIDENTTYPE'] == ' ':
        return 'Unknown'
    else:
        return row['ACCIDENTTYPE']
crash_data['accident_cat'] = crash_data.apply(func, axis=1)

In [36]:
crash_data[['accident_cat','ACCIDENTTYPE']].head()

,accident_cat,ACCIDENTTYPE
0,Unknown,
1,Prop. Damage,Prop. Damage
2,Prop. Damage,Prop. Damage
3,Unknown,
4,Prop. Damage,Prop. Damage


In [41]:
# link between weather and accident type - getting the counts for source and destination linkage 
weather_accidenttype = crash_data[['STREETSEGID','weather_cat','accident_cat']].groupby(['weather_cat','accident_cat']).count().reset_index()
weather_accidenttype.columns = ['weather_cat','accident_cat','count']
print weather_accidenttype
weather_accidenttype.to_csv('weather_accidenttype.csv')

         weather_cat  accident_cat  count
0              Clear      Fatality    109
1              Clear        Injury  20135
2              Clear  Prop. Damage  35953
3              Clear       Unknown  29851
4           Fog/Mist      Fatality      2
5           Fog/Mist        Injury    222
6           Fog/Mist  Prop. Damage    450
7           Fog/Mist       Unknown    339
8               Rain      Fatality      8
9               Rain        Injury   2588
10              Rain  Prop. Damage   4641
11              Rain       Unknown   3524
12  Severe Crosswind      Fatality      1
13  Severe Crosswind        Injury     67
14  Severe Crosswind  Prop. Damage    145
15  Severe Crosswind       Unknown    107
16              Snow      Fatality      2
17              Snow        Injury    297
18              Snow  Prop. Damage    710
19              Snow       Unknown    688
20            others      Fatality      6
21            others        Injury   1089
22            others  Prop. Damage

In [50]:
# get ward data from Alexis
crash_data2 = pd.read_csv('ward_crash_short.csv')
crash_data2.columns = ['MILITARYTIME','STREETSEGI','ward']
crash_data2.head()

,MILITARYTIME,STREETSEGI,ward
0,1254,2356,Ward 3
1,1626,7198,Ward 3
2,1615,10734,Ward 3
3,945,3429,Ward 3
4,2030,927,Ward 3


In [55]:
# classify time
crash_data2['Time_cat'] = "NA"
def func(row):
    if row['MILITARYTIME'] >= 0 and row['MILITARYTIME'] < 0700:
        return 'Overnight'
    elif row['MILITARYTIME'] >= 0700 and row['MILITARYTIME'] < 1000:
        return 'AM_rush'
    elif row['MILITARYTIME'] >= 1000 and row['MILITARYTIME'] < 1200:
        return 'AM_norush'
    elif row['MILITARYTIME'] >= 1200 and row['MILITARYTIME'] < 1600:
        return 'PM_norush'
    elif row['MILITARYTIME'] >= 1600 and row['MILITARYTIME'] < 1800:
        return 'AM_norush'
    elif row['MILITARYTIME'] >= 1800 and row['MILITARYTIME'] < 2200:
        return 'Evening'
    elif row['MILITARYTIME'] >= 2200:
        return 'Overnight'
    else:
        return 'NoTime'
crash_data2['Time_cat'] = crash_data.apply(func, axis=1)
crash_data2.head()

,MILITARYTIME,STREETSEGI,ward,Time_cat
0,1254,2356,Ward 3,PM_norush
1,1626,7198,Ward 3,AM_norush
2,1615,10734,Ward 3,AM_norush
3,945,3429,Ward 3,AM_rush
4,2030,927,Ward 3,Evening


In [56]:
# link between ward and time - getting the counts for source and destination linkage 
ward_time = crash_data2[['STREETSEGI','ward','Time_cat']].groupby(['ward','Time_cat']).count().reset_index()
ward_time.columns = ['NAME','Time_cat','count']
print ward_time
ward_time.to_csv('ward_time.csv')

      NAME   Time_cat  count
0   Ward 1  AM_norush   2257
1   Ward 1    AM_rush   1693
2   Ward 1    Evening   1702
3   Ward 1  Overnight   2069
4   Ward 1  PM_norush   2116
5   Ward 2  AM_norush   6058
6   Ward 2    AM_rush   4649
7   Ward 2    Evening   4570
8   Ward 2  Overnight   4856
9   Ward 2  PM_norush   5452
10  Ward 3  AM_norush   1877
11  Ward 3    AM_rush   1481
12  Ward 3    Evening   1344
13  Ward 3  Overnight    819
14  Ward 3  PM_norush   1830
15  Ward 4  AM_norush   2164
16  Ward 4    AM_rush   1729
17  Ward 4    Evening   1630
18  Ward 4  Overnight   1666
19  Ward 4  PM_norush   2023
20  Ward 5  AM_norush   3706
21  Ward 5    AM_rush   3201
22  Ward 5    Evening   2725
23  Ward 5  Overnight   2780
24  Ward 5  PM_norush   3744
25  Ward 6  AM_norush   3874
26  Ward 6    AM_rush   3304
27  Ward 6    Evening   2834
28  Ward 6  Overnight   2740
29  Ward 6  PM_norush   3788
30  Ward 7  AM_norush   2676
31  Ward 7    AM_rush   2148
32  Ward 7    Evening   2536
33  Ward 7  Ov